# ***1. Доп. ранжирование по вероятности (3 балла)***
Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [182]:
!pip install razdel tqdm

In [183]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
import pandas as pd
import numpy as np
from tqdm import tqdm


In [186]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/spelling/data.zip

!unzip -o data.zip

--2024-12-05 20:29:58--  https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/spelling/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/spelling/data.zip [following]
--2024-12-05 20:29:59--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/spelling/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18787000 (18M) [application/zip]
Saving to: ‘data.zip.2’

data.zip.2          100%[===================>]  17.92M  --.-KB/s    in 0.1s    

2024-

In [188]:
!pip install textdistance
import textdistance
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [191]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [192]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [193]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    # вся эффективноть берется из того, что мы сразу считаем близость
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

In [194]:
def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(topn)

In [195]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)


    return closest

In [196]:
N = sum(vocab.values())

def P(lst, N=N):
  probab_lst=[]
  for word in lst:
    probab_lst.append(vocab[word] / N)
  return probab_lst

In [197]:
result = get_closest_hybrid_match('жукм', X, vec)

In [198]:
def res2df(result=result):
  df = pd.DataFrame(result, columns=['var', 'Similarity'],
                    index=[i for i in range(1,  len(result)+1)])
  df['probability'] = P(df['var'].tolist(), N=N)
  df['combined_probab'] = df.probability*df['Similarity']
  return df.sort_values(by='probability', ascending=False)

In [199]:
res2df(result)

,var,Similarity,probability,combined_probab
3,жук,0.75,2.230723e-05,1.673042e-05
2,жуком,0.80,3.879518e-07,3.103615e-07
1,жукам,0.80,1.939759e-07,1.551807e-07


2. Symspell (7 баллов)
# ***Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:***







Составляется словарь правильных слов

In [200]:
def P1(lst, N=N):

    probab_lst = []
    for word in lst:
      probab_lst.append(vocab[word] / N)

    return probab_lst

Составляется словарь правильных слов

In [201]:

true = open('/content/correct_sents.txt', encoding='utf8').read().splitlines()

In [202]:
vocab_true = re.findall('\w+', corpus.lower())

На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово (!)

In [204]:
def delete_char(word):
  '''функция удаляет 1 букву в слове путем замены ее на пустое значение,
  проходясь циклом по всему слову'''
  if len(word) ==1 or  word.isdigit():
    return []
  else:
    return list(set([word.replace(i, '', 1) for i in word]))

# создаем словарь: правильное значение, слова с пропусками
dic ={}

for word in vocab_true:
  for word_with_mis in delete_char(word):


    dic[word_with_mis] = [word, P1([word])[0]]


Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления

In [205]:
def create_all_var(error_word):
  all_var = delete_char(error_word)
  return [word for word in all_var if word in dic.keys()]

Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово

In [206]:
def best_choice(error_word):
  try:
    lst = create_all_var(error_word)

    true_var_lst = []

    for i in lst:
      true_var_lst.append(dic[i][0])

    true_var_lst = list(set(true_var_lst))

    possible = {}

    for i in true_var_lst:
            possible[i] = dic[i][1]  # Вероятность, которая сейчас является числом

        # Сортируем по вероятности
    sorted_possible = sorted(possible.items(), key=lambda item: item[1], reverse=True)

    return sorted_possible  # Возвращаем отсортированные возможные исправления
  except Exception as e:
        return error_word



In [207]:
print(best_choice('дор'))

[('дом', 9.504820204687272e-05), ('пор', 5.256747500959696e-05), ('гдр', 1.939759225446382e-07)]


In [208]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

Оцените качество полученного алгоритма теми же тремя метриками.

In [209]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [215]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab_true):
            pred_choices = best_choice(pair[1])
            if pred_choices:
                pred = pred_choices[0][0]
            else:
                pred = pair[1]
            cashed[pair[1]] = pred
        else:
            pred = pair[1]

        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|██████████| 915/915 [05:24<00:00,  2.82it/s]


In [216]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8430215107553777
0.05124223602484472
0.039852991845641436
